In [2]:
import numpy as np
import tensorflow as tf
import spektral

In [3]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

/home/abishek/Desktop/Projects/virtual_envs/Graph_ML/lib/python3.6/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [9]:
adj=adj+ np.eye(adj.shape[0]) ## self connection adding identity matrix
features=features.astype('float32')
adj=adj.astype('float32')

In [12]:
features.shape ## 2708 nodes and and each node has 1433 features (Bag of words)

(2708, 1433)

In [14]:
adj.shape ## Shape of adjaceny matrix

(2708, 2708)

In [15]:
labels.shape ## There are seven different types of papers

(2708, 7)

In [20]:
print(np.sum(train_mask))
print(np.sum(test_mask))
print(np.sum(val_mask))

140
1000
500


In [65]:
def masked_softmax_cross_entropy_loss(logits,labels,mask):
    loss=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels)
    mask=tf.cast(mask,dtype=tf.float32)
    mask/=tf.reduce_mean(mask)
    loss*=mask
    return tf.reduce_mean(loss)

def masked_accuracy(logits,labels,mask):
    correct_predictions=tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
    accuracy_all=tf.cast(correct_predictions,tf.float32)
    mask=tf.cast(mask,dtype=tf.float32)
    mask /=tf.reduce_mean(mask)
    accuracy_all *=mask
    
    return tf.reduce_mean(accuracy_all)

In [41]:
## example for reduce_mean and cast
x = tf.constant([True,False,True,False])
x=tf.cast(x,dtype=tf.float32)
print(tf.reduce_mean(x))

tf.argmax([[0.8,0.2],[0.2,0.8]],1)

tf.Tensor(0.5, shape=(), dtype=float32)


In [49]:
def gnn(fts,adj,transform,activation):
    seq_fts=transform(fts)  # HW matrix
    ret_fts=tf.matmul(adj,seq_fts) ## A*HW
    
    return activation(ret_fts) ##sigma(AW)

In [73]:
def train_cora(fts,adj,gnn_fn,units,epochs,lr):
    layer_1=tf.keras.layers.Dense(units)
    layer_2=tf.keras.layers.Dense(7)
    
    def cora_gnn(fts,adj):
        hidden=gnn_fn(fts,adj,layer_1,tf.nn.relu)
        logits=gnn_fn(hidden,adj,layer_2,tf.identity)
        
        return logits
    
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    best_accuracy=0.0
    for epoch in range(epochs+1):
        with tf.GradientTape() as t:
            logits=cora_gnn(fts,adj)
            loss=masked_softmax_cross_entropy_loss(logits,labels,train_mask)
            
        variables=t.watched_variables()
        grads=t.gradient(loss,variables)
        optimizer.apply_gradients((zip(grads,variables)))
        
        logits =cora_gnn(fts,adj)
        val_accuracy=masked_accuracy(logits,labels,val_mask)
        test_accuracy=masked_accuracy(logits,labels,test_mask)
        
        if val_accuracy>best_accuracy:
            best_accuracy = val_accuracy
            print('Epoch',epoch,'| Training loss:',loss.numpy(),'| Val accuracy:',val_accuracy.numpy()
                 ,'| Test accuracy:',test_accuracy.numpy())

In [74]:
train_cora(features,adj,gnn,32,300,0.01)

Epoch 0 | Training loss: 5.2061796 | Val accuracy: 0.17399998 | Test accuracy: 0.18699999
Epoch 1 | Training loss: 6.954589 | Val accuracy: 0.364 | Test accuracy: 0.393
Epoch 2 | Training loss: 4.006788 | Val accuracy: 0.576 | Test accuracy: 0.588
Epoch 3 | Training loss: 1.713075 | Val accuracy: 0.67399997 | Test accuracy: 0.665
Epoch 9 | Training loss: 0.6422431 | Val accuracy: 0.684 | Test accuracy: 0.67
Epoch 10 | Training loss: 0.57476926 | Val accuracy: 0.698 | Test accuracy: 0.68
Epoch 11 | Training loss: 0.5068525 | Val accuracy: 0.70399994 | Test accuracy: 0.693
Epoch 12 | Training loss: 0.4399227 | Val accuracy: 0.71199995 | Test accuracy: 0.704
Epoch 13 | Training loss: 0.37220645 | Val accuracy: 0.71999997 | Test accuracy: 0.71199995
Epoch 21 | Training loss: 0.12966953 | Val accuracy: 0.72 | Test accuracy: 0.72999996
Epoch 31 | Training loss: 0.03954349 | Val accuracy: 0.72199994 | Test accuracy: 0.716
Epoch 33 | Training loss: 0.03339042 | Val accuracy: 0.724 | Test accur

KeyboardInterrupt: 

In [76]:
train_cora(features,tf.eye(adj.shape[0]),gnn,32,300,0.01)

Epoch 0 | Training loss: 1.9566858 | Val accuracy: 0.248 | Test accuracy: 0.237
Epoch 1 | Training loss: 1.6681218 | Val accuracy: 0.35399997 | Test accuracy: 0.329
Epoch 2 | Training loss: 1.4171373 | Val accuracy: 0.41199997 | Test accuracy: 0.39299995
Epoch 3 | Training loss: 1.1599797 | Val accuracy: 0.45599997 | Test accuracy: 0.42799997
Epoch 4 | Training loss: 0.9040318 | Val accuracy: 0.51000005 | Test accuracy: 0.45899996
Epoch 5 | Training loss: 0.674863 | Val accuracy: 0.52 | Test accuracy: 0.48099998
Epoch 6 | Training loss: 0.48830017 | Val accuracy: 0.536 | Test accuracy: 0.49699998
Epoch 7 | Training loss: 0.34675533 | Val accuracy: 0.54200006 | Test accuracy: 0.517
Epoch 8 | Training loss: 0.24566871 | Val accuracy: 0.548 | Test accuracy: 0.51799995
Epoch 10 | Training loss: 0.12986109 | Val accuracy: 0.55399996 | Test accuracy: 0.52599996
Epoch 11 | Training loss: 0.09795472 | Val accuracy: 0.564 | Test accuracy: 0.519


KeyboardInterrupt: 

In [77]:
## Mean pooling --> using degree matrix
deg = tf.reduce_sum(adj,axis=-1)
train_cora(features,adj/deg,gnn,32,300,0.01)

Epoch 0 | Training loss: 1.9391274 | Val accuracy: 0.288 | Test accuracy: 0.341
Epoch 1 | Training loss: 1.7133814 | Val accuracy: 0.39399996 | Test accuracy: 0.431
Epoch 2 | Training loss: 1.4775066 | Val accuracy: 0.5 | Test accuracy: 0.54399997
Epoch 3 | Training loss: 1.2303725 | Val accuracy: 0.64599997 | Test accuracy: 0.689
Epoch 4 | Training loss: 1.001589 | Val accuracy: 0.73999995 | Test accuracy: 0.7679999
Epoch 5 | Training loss: 0.8053624 | Val accuracy: 0.76399994 | Test accuracy: 0.78999996
Epoch 6 | Training loss: 0.64455986 | Val accuracy: 0.778 | Test accuracy: 0.8009999
Epoch 7 | Training loss: 0.5129296 | Val accuracy: 0.784 | Test accuracy: 0.81299984
Epoch 8 | Training loss: 0.4051045 | Val accuracy: 0.78800005 | Test accuracy: 0.8159998


KeyboardInterrupt: 

In [80]:
norm_deg=tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj=tf.matmul(norm_deg,tf.matmul(adj,norm_deg))

train_cora(features,norm_adj,gnn,32,300,0.01)

Epoch 0 | Training loss: 1.9445424 | Val accuracy: 0.42199996 | Test accuracy: 0.48899996
Epoch 1 | Training loss: 1.7715251 | Val accuracy: 0.578 | Test accuracy: 0.625
Epoch 2 | Training loss: 1.5576464 | Val accuracy: 0.64599997 | Test accuracy: 0.679
Epoch 3 | Training loss: 1.3204114 | Val accuracy: 0.692 | Test accuracy: 0.717
Epoch 4 | Training loss: 1.098668 | Val accuracy: 0.714 | Test accuracy: 0.74399996
Epoch 5 | Training loss: 0.8958506 | Val accuracy: 0.75 | Test accuracy: 0.7649999
Epoch 6 | Training loss: 0.714831 | Val accuracy: 0.76000005 | Test accuracy: 0.7759999
Epoch 7 | Training loss: 0.5596876 | Val accuracy: 0.77199996 | Test accuracy: 0.7819999
Epoch 13 | Training loss: 0.114426345 | Val accuracy: 0.774 | Test accuracy: 0.7969999
Epoch 15 | Training loss: 0.06718614 | Val accuracy: 0.77599996 | Test accuracy: 0.79099995
Epoch 16 | Training loss: 0.052305657 | Val accuracy: 0.78 | Test accuracy: 0.78999996


KeyboardInterrupt: 

In [81]:
## 3 layers

In [84]:
def gnn(fts,adj,transform,activation):
    seq_fts=transform(fts)  # HW matrix
    ret_fts=tf.matmul(adj,seq_fts) ## A*HW
    
    return activation(ret_fts) ##sigma(AW)

def train_cora(fts,adj,gnn_fn,units_1,units_2,epochs,lr):
    layer_1=tf.keras.layers.Dense(units_1)
    layer_2=tf.keras.layers.Dense(units_2)
    layer_3=tf.keras.layers.Dense(7)
    
    def cora_gnn(fts,adj):
        hidden_1=gnn_fn(fts,adj,layer_1,tf.nn.relu)
        hidden_2=gnn_fn(hidden_1,adj,layer_2,tf.nn.relu)
        logits=gnn_fn(hidden_2,adj,layer_3,tf.identity)
        
        return logits
    
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    best_accuracy=0.0
    for epoch in range(epochs+1):
        with tf.GradientTape() as t:
            logits=cora_gnn(fts,adj)
            loss=masked_softmax_cross_entropy_loss(logits,labels,train_mask)
            
        variables=t.watched_variables()
        grads=t.gradient(loss,variables)
        optimizer.apply_gradients((zip(grads,variables)))
        
        logits =cora_gnn(fts,adj)
        val_accuracy=masked_accuracy(logits,labels,val_mask)
        test_accuracy=masked_accuracy(logits,labels,test_mask)
        
        if val_accuracy>best_accuracy:
            best_accuracy = val_accuracy
            print('Epoch',epoch,'| Training loss:',loss.numpy(),'| Val accuracy:',val_accuracy.numpy()
                 ,'| Test accuracy:',test_accuracy.numpy())

In [85]:
deg = tf.reduce_sum(adj,axis=-1)
train_cora(features,adj/deg,gnn,64,32,300,0.01)

Epoch 0 | Training loss: 1.9375827 | Val accuracy: 0.324 | Test accuracy: 0.359
Epoch 1 | Training loss: 1.7313331 | Val accuracy: 0.496 | Test accuracy: 0.513
Epoch 3 | Training loss: 1.1579605 | Val accuracy: 0.572 | Test accuracy: 0.578
Epoch 4 | Training loss: 0.8657939 | Val accuracy: 0.658 | Test accuracy: 0.702
Epoch 5 | Training loss: 0.62289214 | Val accuracy: 0.724 | Test accuracy: 0.7519999
Epoch 6 | Training loss: 0.43806252 | Val accuracy: 0.728 | Test accuracy: 0.76199985
Epoch 7 | Training loss: 0.29763174 | Val accuracy: 0.74799997 | Test accuracy: 0.7719999
Epoch 9 | Training loss: 0.12900697 | Val accuracy: 0.74999994 | Test accuracy: 0.7779999


KeyboardInterrupt: 

In [86]:
norm_deg=tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj=tf.matmul(norm_deg,tf.matmul(adj,norm_deg))

train_cora(features,norm_adj,gnn,64,32,300,0.01)

Epoch 0 | Training loss: 1.9433129 | Val accuracy: 0.48399997 | Test accuracy: 0.5
Epoch 1 | Training loss: 1.8097103 | Val accuracy: 0.672 | Test accuracy: 0.69
Epoch 2 | Training loss: 1.6013825 | Val accuracy: 0.72199994 | Test accuracy: 0.714
Epoch 5 | Training loss: 0.81227857 | Val accuracy: 0.762 | Test accuracy: 0.7649999
Epoch 6 | Training loss: 0.5918867 | Val accuracy: 0.77000004 | Test accuracy: 0.7769999
Epoch 7 | Training loss: 0.417977 | Val accuracy: 0.77599996 | Test accuracy: 0.80099994
Epoch 8 | Training loss: 0.28479737 | Val accuracy: 0.786 | Test accuracy: 0.8109998
Epoch 9 | Training loss: 0.18789074 | Val accuracy: 0.79599994 | Test accuracy: 0.8079998


KeyboardInterrupt: 

In [87]:
norm_deg=tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj=tf.matmul(norm_deg,tf.matmul(adj,norm_deg))

train_cora(features,norm_adj,gnn,128,64,300,0.01)

Epoch 0 | Training loss: 1.9480963 | Val accuracy: 0.664 | Test accuracy: 0.644
Epoch 1 | Training loss: 1.7386317 | Val accuracy: 0.73199993 | Test accuracy: 0.7509999
Epoch 2 | Training loss: 1.3894283 | Val accuracy: 0.75399995 | Test accuracy: 0.7819999
Epoch 3 | Training loss: 0.94913226 | Val accuracy: 0.7799999 | Test accuracy: 0.8019998
Epoch 4 | Training loss: 0.54930115 | Val accuracy: 0.786 | Test accuracy: 0.8119998
Epoch 5 | Training loss: 0.2802494 | Val accuracy: 0.78999996 | Test accuracy: 0.8069998
Epoch 6 | Training loss: 0.13407582 | Val accuracy: 0.79199994 | Test accuracy: 0.8039998


KeyboardInterrupt: 